In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torchsummary import summary
import math

In [20]:
# transform = transforms.Compose([transforms.Resize(224),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
#                                 normalize])


normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
transform = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
])



traindata = torchvision.datasets.ImageFolder('./data/trainset/', transform=transform)
testset =torchvision.datasets.ImageFolder('./data/test/', transform=transform)


size = [int(len(traindata)*0.9), len(traindata) - int(len(traindata)*0.9)]

trainset, validset = torch.utils.data.random_split(dataset=traindata, lengths=size)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(validset,
                                          batch_size=16,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=32,
                                        shuffle=True)
classes = ('Cat', 'Dog')

In [2]:
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            transforms.ToTensor(),
            normalize,
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
cuda_available = torch.cuda.is_available()
print (cuda_available)

True


In [14]:
# __all__ = [
#     'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
#     'vgg19_bn', 'vgg19',
# ]


# model_urls = {
#     'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
#     'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
#     'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
#     'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
#     'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
#     'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
#     'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
#     'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
# }


# class VGG(nn.Module):

#     def __init__(self, features, num_classes=10, init_weights=True):
#         super(VGG, self).__init__()
#         self.features = features
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, num_classes),
#         )
#         if init_weights:
#             self._initialize_weights()

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(-1, x.size(0))
# #         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return x

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
#                 m.weight.data.normal_(0, math.sqrt(2. / n))
#                 if m.bias is not None:
#                     m.bias.data.zero_()
#             elif isinstance(m, nn.BatchNorm2d):
#                 m.weight.data.fill_(1)
#                 m.bias.data.zero_()
#             elif isinstance(m, nn.Linear):
#                 m.weight.data.normal_(0, 0.01)
#                 m.bias.data.zero_()


# def make_layers(cfg, batch_norm=False):
#     layers = []
#     in_channels = 3
#     for v in cfg:
#         if v == 'M':
#             layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
#         else:
#             conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
#             if batch_norm:
#                 layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
#             else:
#                 layers += [conv2d, nn.ReLU(inplace=True)]
#             in_channels = v
#     return nn.Sequential(*layers)


# cfg = {
#     'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
#     'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
# }


# def vgg11(pretrained=False, **kwargs):
#     """VGG 11-layer model (configuration "A")

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['A']), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
#     return model



# def vgg11_bn(pretrained=False, **kwargs):
#     """VGG 11-layer model (configuration "A") with batch normalization

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg11_bn']))
#     return model



# def vgg13(pretrained=False, **kwargs):
#     """VGG 13-layer model (configuration "B")

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['B']), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg13']))
#     return model



# def vgg13_bn(pretrained=False, **kwargs):
#     """VGG 13-layer model (configuration "B") with batch normalization

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['B'], batch_norm=True), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg13_bn']))
#     return model



# def vgg16(pretrained=False, **kwargs):
#     """VGG 16-layer model (configuration "D")

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['D']), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg16']))
#     return model



# def vgg16_bn(pretrained=False, **kwargs):
#     """VGG 16-layer model (configuration "D") with batch normalization

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['D'], batch_norm=True), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg16_bn']))
#     return model



# def vgg19(pretrained=False, **kwargs):
#     """VGG 19-layer model (configuration "E")

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['E']), **kwargs)
#     if pretrained:
#         model.load_state_dict(model_zoo.load_url(model_urls['vgg19']))
#     return model



# def vgg19_bn(pretrained=False, **kwargs):
#     """VGG 19-layer model (configuration 'E') with batch normalization

#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     if pretrained:
#         kwargs['init_weights'] = False
#     model = VGG(make_layers(cfg['E'], batch_norm=True), **kwargs)
#     if pretrained:
#         model.load_

In [15]:
# class NNet(nn.Module):
#     def __init__(self):
#         super(NNet, self).__init__()
#         self.conv = nn.Sequential(
#             # Layaer 1
#             nn.Conv2d(in_channels=3, out_channels=64,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),

#             nn.Conv2d(in_channels=64, out_channels=64,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
#             # Layaer 3
#             nn.Conv2d(in_channels=64, out_channels=128,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=128, out_channels=128,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
#             # Layaer 5
#             nn.Conv2d(in_channels=128, out_channels=256,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),

#             nn.Conv2d(in_channels=256, out_channels=256,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=256, out_channels=256,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
#              # Layaer 8
#             nn.Conv2d(in_channels=256, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
        
#             # Layaer 11
#             nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
            
#             nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),            
#         )
#         self.view(-1, 7*7*512)
#         self.net = nn.Linear(7*7*512, 4096)
#         self.net = nn.Linear(4096, 4096)
#         self.net = nn.Linear(4096, 10)
        
        
#     def forward(self, x):
# #         x = self.net(self.conv(x).squeeze())
# #         x = x.view(-1, 7*7*512)
# #         x = self.net = nn.Linear(7*7*512, 4096)
# #         x = self.net = nn.Linear(4096, 4096)
# #         x = self.net = nn.Linear(4096, 10)
#         return self.net(self.conv(x).squeeze())

In [16]:
class VGG16(nn.Module):
    def __init__(self, n_classes):
        super(VGG16, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(7*7*512, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.pool(x)
        x = x.view(-1, 7 * 7 * 512)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x

In [17]:
net = VGG16(2)
# net = vgg16()
# net = NNet()
learning_rate = 1e-3
if cuda_available:
    net.cuda()
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [18]:
summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
            Conv2d-2         [-1, 64, 224, 224]          36,928
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
            Conv2d-5        [-1, 128, 112, 112]         147,584
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
            Conv2d-8          [-1, 256, 56, 56]         590,080
            Conv2d-9          [-1, 256, 56, 56]         590,080
        MaxPool2d-10          [-1, 256, 28, 28]               0
           Conv2d-11          [-1, 512, 28, 28]       1,180,160
           Conv2d-12          [-1, 512, 28, 28]       2,359,808
           Conv2d-13          [-1, 512, 28, 28]       2,359,808
        MaxPool2d-14          [-1, 512,

In [19]:
c = 0
acc = []
for epoch in range(50):
    losses = []
    # Train
    for batch_idx, (images, labels) in enumerate(trainloader):
        if cuda_available:
            images, labels = images.cuda(), labels.cuda()

        optimizer.zero_grad()
#         print (images.shape)
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())

        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    net.eval()
    total = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(validloader):
        if cuda_available:
            images, targets = images.cuda(), labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        accuracy = 100.*(correct/total)
        acc.append(accuracy)

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print (learning_rate)
    print('--------------------------------------------------------------')
    net.train()
    c += 1
    if (c%5 == 0)  and (acc[epoch] <= min(acc[epoch-4:epoch-1])):
        learning_rate *= 0.1

Epoch : 0 Loss : 6.909 


KeyboardInterrupt: 